# data_crud
this notebook is for:
1. gathering all data for my personal recording system to a local SQLite db
2. upload to the railway.app hosted PostgreSQL
3. { add fun thing todo }

In [1]:
import os
import sqlite3
import datetime
import psycopg2
import subprocess
import pandas as pd
import geopy.distance
import plotly.express as px
from os.path import expanduser
from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)

engine_staging = create_engine('sqlite:///staging.db', echo=False)
engine = create_engine(os.environ.get('POSTGRES_URI',''))


#display(os.environ.get('imap_gmail_password',''))
#print()
#subprocess.call(['conda', 'env', 'config', 'vars', 'list'])

# railway.app PostgreSQL database tables
data we have in the 'finished' state

In [2]:
pd.read_sql_query("""SELECT 
    pgClass.relname   AS tableName, 
    pgClass.reltuples AS rowCount
    FROM pg_class pgClass
INNER JOIN pg_namespace pgNamespace ON (pgNamespace.oid = pgClass.relnamespace)
WHERE pgNamespace.nspname NOT IN ('pg_catalog', 'information_schema') AND pgClass.relkind='r'
AND pgNamespace.nspname = 'public'
ORDER BY tablename
""", con=engine)

tablename   rowcount
0                fitbit_sleep       -1.0
1             fitbit_sleeping       -1.0
2                   heartrate  1984110.0
3           heartrate_per_day      194.0
4                     running   129040.0
5             running_per_day       67.0
6           screentime_iphone    15523.0
7              screentime_mac    11935.0
8  self_reported_stress_score       -1.0
9                work_2024_12       -1.0

# DONE from file
# screentime mac+iphone
this is good as many things integrate into apple health
requirements:
1. manually copy ```knowledgeC.db``` from ```~/Library/Application Support/Knowledge/knowledgeC.db```
2. https://felixkohlhas.com/projects/screentime/

In [ ]:
def query_database():
    knowledge_db = expanduser("knowledgeC.db")
    with sqlite3.connect(knowledge_db) as con:
        cur = con.cursor()
        query = """
        SELECT
            ZOBJECT.ZVALUESTRING AS "app", 
            (ZOBJECT.ZENDDATE - ZOBJECT.ZSTARTDATE) AS "usage",
            (ZOBJECT.ZSTARTDATE + 978307200) as "start_time", 
            (ZOBJECT.ZENDDATE + 978307200) as "end_time",
            (ZOBJECT.ZCREATIONDATE + 978307200) as "created_at", 
            ZOBJECT.ZSECONDSFROMGMT AS "tz",
            ZSOURCE.ZDEVICEID AS "device_id",
            ZMODEL AS "device_model",
            ZSTREAMNAME as "streamname"
        FROM
            ZOBJECT LEFT JOIN ZSTRUCTUREDMETADATA ON ZOBJECT.ZSTRUCTUREDMETADATA = ZSTRUCTUREDMETADATA.Z_PK 
            LEFT JOIN ZSOURCE ON ZOBJECT.ZSOURCE = ZSOURCE.Z_PK 
            LEFT JOIN ZSYNCPEER ON ZSOURCE.ZDEVICEID = ZSYNCPEER.ZDEVICEID
        ORDER BY
            ZSTARTDATE DESC
        """ # WHERE ZSTREAMNAME = "/app/usage"
        cur.execute(query)
        return cur

data = query_database()
df = pd.DataFrame(data.fetchall(), columns=[description[0] for description in data.description])

df[df['device_id'].notnull()].to_sql(f"screentime_iphone", index=False, if_exists='replace', con=engine)
df[~df['device_id'].notnull()].to_sql(f"screentime_mac", index=False, if_exists='replace', con=engine)


# DONE from file
# running
this is good as many things integrate into apple health

In [11]:
import os
import gpxpy
import geopy.distance
import pandas as pd

# Load gpx.
DIR = 'Downloads/apple_health_export/workout-routes/'
dfs = []
for file in os.listdir(DIR):
    print(file)
    gpx_path = f"{DIR}{file}"
    with open(gpx_path) as f:
        gpx = gpxpy.parse(f)
    points = []
    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time': p.time,
                'latitude': p.latitude,
                'longitude': p.longitude,
                'elevation': p.elevation,
            })
    df = pd.DataFrame.from_records(points)
    df['run_id'] = file.split('.')[0]
    # Cumulative distance.
    coords = [(p.latitude, p.longitude) for p in df.itertuples()]
    df['distance'] = [0] + [geopy.distance.distance(from_, to).m for from_, to in zip(coords[:-1], coords[1:])]
    df['cumulative_distance'] = df.distance.cumsum()
    
    # Timing.
    df['duration'] = df.time.diff().dt.total_seconds().fillna(0)
    df['cumulative_duration'] = df.duration.cumsum()
    df['pace_metric'] = pd.Series((df.duration / 60) / (df.distance / 1000)).bfill()
    
    dfs.append(df)


from numpy_ext import rolling_apply
import matplotlib.pyplot as plt

df = pd.concat(dfs, axis=0)
print(df.info())
df.sort_values('time', inplace=True)


#def rolling_metric_pace(duration, distance):
#    return (duration.sum() / 60) / (distance.sum() / 1000)
#x = df.cumulative_distance
#y = rolling_apply(rolling_metric_pace, 100, df.duration.values, df.distance.values)
#plt.plot(x, y)


df['cumulative_distance'] = pd.to_numeric(df['cumulative_distance'])
df.to_sql(f"running", index=False, if_exists='replace', con=engine)

ddf = pd.DataFrame(
    df.groupby([df['time'].dt.date])['cumulative_distance'].max()
)
ddf.to_sql(f"running_per_day", index=False, if_exists='replace', con=engine)


route_2024-08-22_4.19pm.gpx
route_2024-11-17_11.28am.gpx
route_2025-01-01_1.28pm.gpx
route_2024-09-10_5.37pm.gpx
route_2024-09-07_8.49am.gpx
route_2024-07-27_7.00pm.gpx
route_2024-06-30_3.49pm.gpx
route_2024-09-11_7.13pm.gpx
route_2024-06-19_5.37pm.gpx
route_2024-05-20_6.48pm.gpx
route_2024-09-08_5.38pm.gpx
route_2024-06-07_8.37pm.gpx
route_2024-06-02_4.10pm.gpx
route_2024-10-06_1.04pm.gpx
route_2024-12-08_10.33am.gpx
route_2025-01-04_10.54am.gpx
route_2024-06-07_8.56pm.gpx
route_2024-11-18_12.29pm.gpx
route_2024-11-27_8.19pm.gpx
route_2024-09-28_1.38pm.gpx
route_2024-11-06_8.05pm.gpx
route_2025-01-04_4.43pm.gpx
route_2024-07-11_8.46pm.gpx
route_2024-07-31_7.46pm.gpx
route_2024-09-10_6.18pm.gpx
route_2024-06-19_3.52pm.gpx
route_2024-07-19_4.21pm.gpx
route_2024-12-25_11.28am.gpx
route_2024-11-05_4.36pm.gpx
route_2024-12-17_5.46pm.gpx
route_2024-09-06_4.49pm.gpx
route_2024-09-18_4.51pm.gpx
route_2024-11-15_5.22pm.gpx
route_2024-10-08_5.38pm.gpx
route_2024-10-27_11.44am.gpx
route_2024-06-

68

# DONE from file
# work time and should time
from personalwolke extract that is named ```work_yyyy_mm.xlsx```

In [48]:
DIR = 'Downloads'
for file in os.listdir(DIR):
    if 'work' in file:
        print(file)
        df = pd.read_excel(f"{DIR}/{file}")
        df.info()
        df.to_sql(f"{file.split('.')[0]}", index=False, if_exists='replace', con=engine)

work_2024_12.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Group                   41 non-null     object 
 1   Name                    41 non-null     object 
 2   Date                    41 non-null     object 
 3   FB                      0 non-null      float64
 4   Bookings                0 non-null      float64
 5   from                    25 non-null     object 
 6   to                      25 non-null     object 
 7   FE                      0 non-null      float64
 8   Absence Reason          10 non-null     object 
 9   Daily target time       32 non-null     object 
 10  Daily working time      23 non-null     object 
 11  Time-Balance            23 non-null     object 
 12  Monthly sum             23 non-null     object 
 13  autom. break deduction  23 non-null     object 
dtypes: float64(3), object(11)


# DONE from file
# heartrate
from fitbit whole data export

In [70]:
DIR = 'Downloads/MyFitbitData/AbelMoricz/Physical Activity'
heart_rates = []
for file in os.listdir(DIR):
    if file.startswith('heart_rate'):
        if ('2021' not in file) and ('2022' not in file) and ('2023' not in file):
            print(file)
            heart_rate = pd.read_json(f"{DIR}/{file}")
            #print(heart_rate.info())
            heart_rates.append(heart_rate)
df = pd.concat(heart_rates)
df[['bpm','confidence']] = pd.json_normalize(df['value'])
df.drop(columns='value',inplace=True)
df.to_sql(f"heartrate", index=False, if_exists='replace', con=engine)

ddf = pd.DataFrame(
    df.groupby([df['dateTime'].dt.date])['bpm'].mean()
)
ddf.to_sql(f"heartrate_per_day", index=True, if_exists='replace', con=engine)

heart_rate-2024-10-18.json
heart_rate-2024-12-25.json
heart_rate-2024-09-02.json
heart_rate-2024-06-27.json
heart_rate-2024-09-14.json
heart_rate-2024-11-23.json
heart_rate-2024-08-15.json
heart_rate-2024-07-26.json
heart_rate-2024-10-22.json
heart_rate-2024-11-19.json
heart_rate-2024-08-03.json
heart_rate-2024-07-30.json
heart_rate-2024-10-14.json
heart_rate-2024-07-10.json
heart_rate-2024-08-23.json
heart_rate-2024-09-18.json
heart_rate-2024-07-06.json
heart_rate-2024-10-02.json
heart_rate-2024-06-07.json
heart_rate-2024-08-19.json
heart_rate-2024-11-15.json
heart_rate-2024-12-05.json
heart_rate-2024-09-22.json
heart_rate-2024-06-11.json
heart_rate-2024-06-10.json
heart_rate-2024-09-23.json
heart_rate-2024-12-04.json
heart_rate-2024-11-14.json
heart_rate-2024-06-06.json
heart_rate-2024-08-18.json
heart_rate-2024-10-03.json
heart_rate-2024-09-19.json
heart_rate-2024-07-07.json
heart_rate-2024-08-22.json
heart_rate-2024-07-11.json
heart_rate-2024-10-15.json
heart_rate-2024-07-31.json
h

# BANK csv to db upload including minimal editing
.csv exports from ELBA app on my iPhone air dropped to my Mac <br/>

In [4]:
DIR = '../Downloads'
for file in os.listdir(DIR):
    if 'Umsatzliste_AT94' in file:
        df = pd.read_csv( f"{DIR}/{file}", delimiter=';', names=['date','purpose', 'd','amount','euro','timestamp'], )
        df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d.%m.%Y %H:%M:%S:%f')
        
        df.to_sql(f"bank_{df['date'].dt.month.max()}_{df['date'].dt.year.max()}", if_exists='replace', con=engine, index=False)

In [28]:
df = pd.read_sql_query("SELECT table_name FROM information_schema.tables WHERE table_schema='public'",con=engine)
dfs = []
for table_name in df['table_name']:
    if 'bank_' in table_name:
        ddf = pd.read_sql_query(f"SELECT * FROM {table_name}",con=engine)
        dfs.append(ddf)
        
expenses_income = pd.concat(dfs, axis=0, ignore_index=True)
expenses_income[['A', 'B']] = expenses_income['amount'].str.split(',', n=1, expand=True)
expenses_income['A'] = expenses_income['A'].str.replace('.','', regex=False)
expenses_income['amount'] = expenses_income['A'] + '.' + expenses_income['B']
expenses_income['amount'] = pd.to_numeric(expenses_income['amount'])

#expenses_income = expenses_income[~expenses_income['purpose'].str.contains('Recipient IBAN: AT933200000013349683')]
#expenses_income = expenses_income[~expenses_income['purpose'].str.contains('Client IBAN: AT933200000013349683')]
#expenses_income = expenses_income[~expenses_income['purpose'].str.contains('AT260100000005504068')] # finanzampt
#expenses_income['balance'] = expenses_income['amount'].cumsum()
expenses_income.sort_values('date')#[0:5]
expenses_income.to_sql(f"bank_checking", if_exists='replace', con=engine, index=False)

bank_10_2024
bank_9_2024
bank_7_2024
bank_11_2024
bank_8_2024


295

In [2]:
fig = px.scatter(x=expenses_income['date'], y=expenses_income['balance'], title='balance')

NameError: name 'expenses_income' is not defined

# archive ---

# email access to abel.moricz.2@gmail.com and abel.moricz@gmail.com

In [2]:
import os
import email
import imaplib  

print(os.environ.get('imap_gmail_password','aaa'))
mail = imaplib.IMAP4_SSL('imap.gmail.com', 993)  
#mail.login('abel.moricz.2@gmail.com', 'kxjo pqmm ofjb uqvt')
mail.login('abel.moricz.2@gmail.com', os.environ.get('imap_gmail_password',''))


all_mail = float(mail.select('Inbox')[1][0].decode("utf-8")) - 2
print(type(all_mail))
print(all_mail)
mail.search(None,'UnSeen')

aaa


error: b'Empty username or password. l12mb34095790wms'

# reading a .pdf line-by-line and outputting it into a .txt file

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader('Downloads/homo_deus_a_brief_history_of_tomorrow_pdf.pdf')
page = reader.pages[0]
text_arr = []
for page in reader.pages[213:230]: # relevant section shared by Rebekka
    extracted_text = page.extract_text()
    text = text + extracted_text
    text_arr.append(extracted_text)
text = text.replace('\n', ' ').replace('\r', '').replace('. ', '. \n')
with open("data_religion.txt", "w") as text_file:
    text_file.write(text)
text_file